In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import ast

In [3]:
import numpy as np
import pandas as pd
 
#data visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
 
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
 
#to avoid warnings
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset

In [5]:
import os
from collections import Counter
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import spacy
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    f1_score,
    multilabel_confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import normalize
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

In [6]:
df = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test_labels.csv')
df_testcomments = pd.read_csv('dataset/test.csv')

In [7]:
# obtain class labels of the dataset
class_labels = list(df.columns[2:])
class_labels

# remove rows with -1 from df_test as it is not used for scoring
print(f'Before removing -1: {df_test.shape}')
for class_label in class_labels:
    df_test = df_test[df_test[class_label] != -1]
print(f'After removing -1: {df_test.shape}')

# merge df_test and df_testcomments on id
df_test = pd.merge(df_test, df_testcomments, on='id', how='left')
# rearraange columns to be the same as df
df_test = df_test[['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

Before removing -1: (153164, 7)
After removing -1: (63978, 7)


In [8]:
# Visualizing the class distribution of the 'label' column
column_labels = df.columns.tolist()[2:]
label_counts = df[column_labels].sum().sort_values()

In [9]:
# Get label for train and test data
y = df[class_labels]
y_test = df_test[class_labels]

In [10]:
df_combined = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined.csv')
df_combined_test = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined_test.csv')

In [11]:
df_combined.head()

,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"['explanation', 'edit', 'username', 'hardcore'...",0,0,0,0,0,0
1,"['aww', 'match', 'background', 'colour', 'I', ...",0,0,0,0,0,0
2,"['hey', 'man', 'I', 'try', 'edit', 'war', 'guy...",0,0,0,0,0,0
3,"['real', 'suggestion', 'improvement', 'wonder'...",0,0,0,0,0,0
4,"['sir', 'hero', 'chance', 'remember', 'page']",0,0,0,0,0,0


In [12]:
for i in range(len(df_combined['comment'])):
    df_combined["comment"][i] = ast.literal_eval(df_combined["comment"][i])

In [13]:
df_combined.head()

,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"[explanation, edit, username, hardcore, metall...",0,0,0,0,0,0
1,"[aww, match, background, colour, I, seemingly,...",0,0,0,0,0,0
2,"[hey, man, I, try, edit, war, guy, constantly,...",0,0,0,0,0,0
3,"[real, suggestion, improvement, wonder, sectio...",0,0,0,0,0,0
4,"[sir, hero, chance, remember, page]",0,0,0,0,0,0


In [14]:
df_combined_test.head()

,comment,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"['thank', 'understanding', 'I', 'think', 'high...",0,0,0,0,0,0
1,"['dear', 'god', 'site', 'horrible']",0,0,0,0,0,0
2,"['somebody', 'invariably', 'try', 'add', 'Reli...",0,0,0,0,0,0
3,"['right', 'type', 'type', 'institution', 'need...",0,0,0,0,0,0
4,"['add', 'new', 'product', 'list', 'sure', 'rel...",0,0,0,0,0,0


In [15]:
for i in range(len(df_combined_test['comment'])):
    df_combined_test["comment"][i] = ast.literal_eval(df_combined_test["comment"][i])

In [17]:
train = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Dataset/train.csv')
test = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Dataset/test.csv')
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [18]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'


def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word

EMBEDDING_FILE = '/Users/ved/Downloads/crawl-300d-2M.vec'

train = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Dataset/train.csv')
test = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Dataset/test.csv')

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

X_train1=[]
X_test1=[]
#Function call to lemmatize X_train and X_test
print ("Train data lemmatization begins")
X_train1 = df_combined['comment']
X_test1 = df_combined_test['comment']
print ("Test data lemmatization ends")

max_features = 30000
maxlen = 100
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
X_train = tokenizer.texts_to_sequences(X_train1)
X_test = tokenizer.texts_to_sequences(X_test1)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)


def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.4)(x)
    x = Bidirectional(GRU(80, return_sequences=True,activation='relu', dropout=0.3, recurrent_dropout=0.))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()


batch_size = 32
epochs = 2

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=2)


y_pred = model.predict(x_test, batch_size=1024)

Train data lemmatization begins
Test data lemmatization ends
Epoch 1/2

 ROC-AUC - epoch: 1 - score: 0.985382 

4738/4738 - 574s - loss: 0.0530 - accuracy: 0.9524 - val_loss: 0.0479 - val_accuracy: 0.9531 - 574s/epoch - 121ms/step
Epoch 2/2

 ROC-AUC - epoch: 2 - score: 0.986353 

4738/4738 - 914s - loss: 0.0427 - accuracy: 0.9342 - val_loss: 0.0472 - val_accuracy: 0.9368 - 914s/epoch - 193ms/step
63/63 [==============================] - 33s 518ms/step


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 100, 300)             9000000   ['input_1[0][0]']             
                                                                                                  
 spatial_dropout1d (Spatial  (None, 100, 300)             0         ['embedding[0][0]']           
 Dropout1D)                                                                                       
                                                                                                  
 bidirectional (Bidirection  (None, 100, 160)             183360    ['spatial_dropout1d[0][0]'

In [23]:
model.weights

[<tf.Variable 'embedding/embeddings:0' shape=(30000, 300) dtype=float32, numpy=
 array([[-0.0548993 ,  0.04768392, -0.05953617, ..., -0.00524814,
          0.06076215,  0.01757216],
        [-0.44426027,  0.03004772, -0.07044135, ..., -0.13697036,
         -0.03995588, -0.04494008],
        [ 0.2988802 ,  0.08662564, -0.23842423, ..., -0.05422498,
          0.2204351 ,  0.04257494],
        ...,
        [ 0.09468539,  0.28709984, -0.5373384 , ..., -0.04616423,
          0.39081943,  0.17720369],
        [-0.78174424,  0.19033286, -0.31379896, ...,  0.03872271,
          0.24832298, -0.26728046],
        [-0.00711457,  0.01299114, -0.01878541, ...,  0.01887535,
          0.02038292, -0.00276587]], dtype=float32)>,
 <tf.Variable 'bidirectional/forward_gru/gru_cell/kernel:0' shape=(300, 240) dtype=float32, numpy=
 array([[-0.17735055, -0.03257459, -0.03579131, ..., -0.06843946,
          0.11694104, -0.08128407],
        [ 0.09530988,  0.00214754,  0.11595257, ...,  0.06049967,
         -

In [22]:
# Save the model architecture and weights to a file
model.save('GRU.h5')